한번에 전체 데이터를 불러온 후에 모델 학습을 진행 한다면 메모리가 터질 수 있음

따라서 배치마다 학습시킬 데이터를 불러와 모델을 학습시키기 위하여 배치 단위로 데이터 셋을 만듬

또한 기존의 폴더 단위에 멜 데이터의 경우 한번 학습시 128번의 데이터를 불러와야 하기 때문에

모델의 학습 속도 측면에서 손해를 불러와 메모리 효율성과 모델 학습 속도 증진을 위하여 배치 단위로 데이터셋을 만듬

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import math
import pickle
import gc

import warnings
warnings.filterwarnings(action='ignore')

data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Data/'
mel_data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Mel_Data/'
batch_data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Batch_Data/'

In [ ]:
song_meta_df = pd.read_json(data_dir + 'song_meta_data_v3.json')
song_meta_df = song_meta_df.sort_values('id')

In [ ]:
song_id_li = song_meta_df['id'].tolist()
del song_meta_df
gc.collect()

In [ ]:
file_name_dic = {}
for song_id in tqdm(song_id_li):
    if math.floor(song_id / 1000) not in file_name_dic:
        file_name_dic[math.floor(song_id / 1000)] = [song_id % 1000]
    else:
        file_name_dic[math.floor(song_id / 1000)] += [song_id % 1000]

    # # 디버깅
    # if math.floor(song_id / 1000) not in file_name_dic:
    #     file_name_dic[math.floor(song_id / 1000)] = [song_id]
    # else:
    #     file_name_dic[math.floor(song_id / 1000)] += [song_id]

In [ ]:
import copy

file_name_li = sorted(list(file_name_dic.keys()))
mel_li = []
batch_mel_spec_li = []
total = 0
batch_name = 1

# # 디버깅
# batch_mel_spec_dic = {}

for file_name in tqdm(file_name_li):
    # mel 불러오기
    with open(mel_data_dir + f'{file_name}.pickle', 'rb') as f:
        mel_pic = pickle.load(f)

    idx_li = file_name_dic[file_name]
    total += len(idx_li)

    # 메모리 정리
    mel_li.append(copy.deepcopy(mel_pic[idx_li]))
    del mel_pic
    del f
    gc.collect()

    # # 디버깅
    # mel_li.append(copy.deepcopy(idx_li))

    if total >= 128:
        mel_spec = np.concatenate(mel_li, axis = 0)
        while total >= 128:
            batch_mel_spec = mel_spec[:128]

            # 배치 저장
            np.save(batch_data_dir + f'{batch_name}.npy', batch_mel_spec)
            
            # 데이터 확인
            print(f'batch_name : {batch_name}, size: ', end = '')
            print(batch_mel_spec.shape, end = ', ')
            print(f'mel.max(): ', end = '')
            print(batch_mel_spec.max())

            # # 디버깅
            # batch_mel_spec_dic[batch_name] = batch_mel_spec

            mel_spec = mel_spec[128:]
            total -= 128
            batch_name += 1

        mel_li = [mel_spec]
    
    else: continue

# 나머지 데이터 저장
batch_mel_spec = np.concatenate(mel_li, axis = 0)
np.save(batch_data_dir + f'{batch_name}.npy', batch_mel_spec)

# # 디버깅
# batch_mel_spec = np.concatenate(mel_li, axis = 0)
# batch_mel_spec_dic[batch_name] = batch_mel_spec

In [ ]:
# 배치크기 + 1 이 만든 총 배치 크기가 됨
print(f'배치 크기: {len(song_id_li) // 128}, 나머지 데이터 크기: {len(song_id_li) % 128}')

In [ ]:
print(f'만든거 - 총 배치 크기: {batch_name}, 나머지 데이터 크기: {len(batch_mel_spec)}')